# Use regression model to locate the landmark point

## Load data

In [ ]:
import pandas as pd
import numpy as np

ROI_CT_path = '../Resources/ROI_CT.xlsx'
ROI_MR_path = '../Resources/ROI_MR.xlsx'

roi_CT = pd.read_excel(ROI_CT_path, index_col=[0, 1], header=[0, 1])

In [ ]:
pname_list = list(set([x for (x, y) in roi_CT.index]))
pname_list.sort()

print(pname_list)

In [ ]:
pixel_space = [0.15, 0.15, 0.15]
data_path_base = "/Volumes/ExternalDis/PhD/Project/Date/CT_Augmented/"

p_name = pname_list[0]
data_path = data_path_base + p_name + "_aug_0.npy"
volume_data = np.load(data_path, allow_pickle=True).item()

In [ ]:
import functions

functions.show_pts(volume_data.get("volume"), volume_data.get("pts"), pixel_space)

## Try regression model

In [ ]:
import numpy as np

x = np.array(
    [56755.72171242, 44178.04737774, 40991.80813814, 8814.00098681, 43585.51198178, 13574.17183072, 6175.8760297,
     17829.69832237, 53254.85637009, 17522.23018625, 42784.69836164, 36638.18492916, 41086.72967373, 18167.77372717,
     12706.89121489, 52564.42917946, 61995.42280258, 35776.79516181, 30230.22630213, 34524.46986093, 13774.60527391,
     14258.22933451, 101376.49657434, 9616.64500569, 45175.23189338, 38615.99518491, 74355.51585756, 12578.49547344,
     19242.3664711, 16310.988409, 20881.76692993, 5734.63362915, 25732.01836475, 51545.48360953, 82081.59716162,
     11006.2497364, 44974.83187718, 56839.38177423])
y = np.array(
    [7.3, 7.1, 6.9, 6.4, 7.4, 6.5, 6.3, 6.7, 7.6, 5.7, 7.6, 6.5, 7.0, 5.4, 5.6, 7.5, 7.0, 7.2, 6.0, 5.9, 5.9, 5.9, 6.9,
     6.5, 7.4, 7.3, 7.6, 6.1, 5.4, 6.2, 5.9, 4.7, 6.3, 7.3, 7.5, 5.5, 6.8, 6.9])

### 1. Polyfit of NumPy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

p = np.poly1d(np.polyfit(x, y, 1))
print(p)
x_line = np.linspace(np.amin(x), np.amax(x), 200)
plt.scatter(x, y)
plt.plot(x_line, p(x_line))
plt.show()

### 2. Linregress of SciPy

In [ ]:
import scipy as sc
from scipy import stats

regr_results = sc.stats.linregress(x, y)
print(regr_results)

### 3. OLS and ols of statsmodels

In [ ]:
import statsmodels.api as sm

x = sm.add_constant(x) # Adds an intercept term to the simple linear regression formula
lin_model = sm.OLS(y, x)
reg_results = lin_model.fit()
print(regr_results.params)

### 4. LinearRegression of scikit-learn

In [ ]:
from sklearn import linear_model

lin_model = linear_model.LinearRegression(fit_intercept=True)
lin_model.fit(x.reshape(-1, 1), y)

In [ ]:
(slope, intercept) = (lin_model.coef_[0], lin_model.intercept_)
print(slope, intercept)

### 5. regplot of seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))
sns.regplot(x = x, y = y, ci=95, order=1,line_kws={'label': 'Linear regression line: $Y(X)=5.74+2.39\cdot 10^{-5} X$', 'color': 'm'}, seed=1,truncate=False, label="Original data")
ax.set_xlabel("GDP per capita 2015 (USD)")
ax.set_ylabel("Life Satisfaction Value")
ax.set_xticks([1000, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000])
ax.set_yticks(np.arange(3.0, 10.5, 0.5))
ax.legend(loc="upper left")

# Try some DNN models

## Load data from .mat --- train, validate, test.

In [ ]:
from os import listdir
from os.path import isfile, join
import numpy as np
import h5py

In [ ]:

# e.g. AZ_17017030_AugVol_1.mat
X_train_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Train/Input/"
# e.g. AZ_17017030_AugPts_1.mat
Y_train_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Train/Output/"

X_train_files = [f for f in listdir(X_train_base_path) if isfile(join(X_train_base_path, f))]

X_train = []
Y_train = []
for X_train_file in X_train_files:
    X_train_file_path = join(X_train_base_path, X_train_file)
    Y_train_file_path = join(Y_train_base_path, X_train_file.replace("AugVol", "AugPts"))
    file_vol = h5py.File(X_train_file_path, 'r')
    file_pts = h5py.File(Y_train_file_path, 'r')
    load_mat_vol = file_vol.get('rescaled_aug_vol')
    load_mat_pts = file_pts.get('rescaled_aug_pts')
    X_train.append(np.array(load_mat_vol).T)
    Y_train.append(np.array(load_mat_pts).reshape(3,4).T)
    file_vol.close()
    file_pts.close()

In [ ]:
print(np.shape(X_train))
print(np.shape(Y_train))

In [ ]:
X_train_reshape = np.asarray(X_train).reshape(700, 170*170*30)

In [ ]:
print(np.asarray(Y_train)[:, 0, :].shape)
print(X_train_reshape.shape)

In [ ]:
import tensorflow as tf
from tensorflow import keras

dataset = tf.data.Dataset.from_tensor_slices((X_train_reshape, Y_train))
dataset = dataset.shuffle(buffer_size=1400).batch(2)

In [ ]:
for step, (x, y) in enumerate(dataset):
    print("step: ", step)
    print("x shape: ", x.shape, type(x))
    print("y shape: ", y.shape, type(y))
    print(y[:, 0, :].shape)
    break

In [ ]:
class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


In [ ]:
class MyMap(keras.layers.Layer):
    """y = x .* w"""

    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], 1),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        m = tf.math.multiply(inputs, self.w)
        m = tf.reshape(m, (inputs.shape[0], 170, 170, 30))
        m = tf.math.reduce_sum(m, 3)
        m = tf.reshape(m, (inputs.shape[0], 170*170))
        return m


In [ ]:
class SimpleMLP(keras.layers.Layer):
    """Simple stack of Linear layers."""

    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.linear_1 = Linear(1020)
        #self.linear_2 = Linear(512)
        self.linear_3 = Linear(3)

    def call(self, inputs):
        x = self.linear_1(inputs)
        #x = self.linear_2(x)
        return self.linear_3(x)

In [ ]:
class TestMLP(keras.layers.Layer):
    """Linear layers with some architecture."""

    def __init__(self):
        super(TestMLP, self).__init__()
        self.my_map = MyMap(170*170)
        self.linear_1 = Linear(1020)
        self.linear_2 = Linear(512)
        self.linear_3 = Linear(3)

    def call(self, inputs):
        x = self.my_map(inputs)
        x = self.linear_1(x)
        x = self.linear_2(x)
        return self.linear_3(x)

mlp = SimpleMLP()

loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-15)

# Create a training step function.

#@tf.function  # Make it fast.
def train_on_batch(x,y):
    with tf.GradientTape() as tape:
        y_pre = mlp(x)
        loss = loss_fn(y[:,0,:], y_pre)
        gradients = tape.gradient(loss, mlp.trainable_weights)
    optimizer.apply_gradients(zip(gradients, mlp.trainable_weights))
    return loss


# Iterate over the batches of the dataset.
for step, (x, y) in enumerate(dataset):
    loss = train_on_batch(x, y)
    if step % 1 == 0:
        print("Step:", step, "Loss:", float(loss))